In [1]:
%load_ext autoreload
%autoreload 2
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.tree import plot_tree

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from hummingbird.ml import convert

from random_forest_pytorch.random_forest_gemm import RandomForestGEMM

In [2]:
# Create RF
X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)

RandomForestClassifier(max_depth=2, random_state=0)

In [3]:
rf_np = RandomForestGEMM(clf, "numpy")
rf_pt = RandomForestGEMM(clf, "torch")
rf_pt_cuda = RandomForestGEMM(clf, "torch", "cuda")

rf_hb = convert(clf, "pytorch")
rf_hb_cuda = convert(clf, "pytorch").to("cuda")

X_pt = torch.Tensor(X)
X_pt_cuda = X_pt.cuda()

In [4]:
%timeit clf.predict(X)

11.1 ms ± 24.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%timeit rf_np.predict(X)

4.92 ms ± 297 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%timeit rf_pt.predict(X_pt)

990 µs ± 68.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
%timeit rf_pt_cuda.predict(X_pt_cuda)

377 µs ± 359 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
%timeit rf_hb.predict(X_pt)

713 µs ± 7.44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
%timeit rf_hb_cuda.predict(X_pt_cuda)

352 µs ± 2.19 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
np.all(rf_pt_cuda.predict(X_pt_cuda).cpu().numpy() == clf.predict(X))

True

In [1]:
11.1*1000 / 377

29.44297082228117

In [2]:
377 / 352

1.0710227272727273

### Profiling avec code customisé (cf ``utils.py``)

In [12]:
from random_forest_pytorch.utils import profile_command, analyze_stack

In [14]:
pdf = pd.read_pickle("pdf.pickle")

In [15]:
pdf.sort_values("cuda", ascending=False).head()["stack"].values[0]

['/home/jovyan/work/ELTDM/eltdm_project/random_forest_pytorch/random_forest_gemm.py(113): vote',
 '/home/jovyan/work/ELTDM/eltdm_project/random_forest_pytorch/random_forest_gemm.py(117): predict',
 'run_profile.py(26): wrapper',
 '/home/jovyan/work/ELTDM/eltdm_project/random_forest_pytorch/utils.py(70): profile_command',
 'run_profile.py(28): <module>']

In [16]:
pdf.sort_values("cuda", ascending=False).head(n=20)

,name,cpu,cuda,stack
3402,aten::_cat,268.960,2224.218750,[/home/jovyan/work/ELTDM/eltdm_project/random_...
3200,aten::stack,116.082,1625.882812,[/home/jovyan/work/ELTDM/eltdm_project/random_...
1,aten::mm,711.122,753.664001,[/home/jovyan/work/ELTDM/eltdm_project/random_...
1422,aten::mm,47.186,306.976562,[/home/jovyan/work/ELTDM/eltdm_project/random_...
2766,aten::mm,60.302,304.351562,[/home/jovyan/work/ELTDM/eltdm_project/random_...
750,aten::mm,55.349,293.759766,[/home/jovyan/work/ELTDM/eltdm_project/random_...
987,aten::mm,93.551,285.568359,[/home/jovyan/work/ELTDM/eltdm_project/random_...
1595,aten::mm,42.811,285.503906,[/home/jovyan/work/ELTDM/eltdm_project/random_...
2093,aten::matmul,37.512,256.226562,[/home/jovyan/work/ELTDM/eltdm_project/random_...
53,aten::eq,42.337,255.296143,[/opt/conda/lib/python3.8/site-packages/torch/...


In [17]:
analyze_stack(pdf.sort_values("cuda", ascending=False).head()["stack"].values[0])

,path,line,top,func,content
0,/home/jovyan/work/ELTDM/eltdm_project/random_f...,113,class RandomForestGEMM:,vote,self.n_trees = len(self.trees)
1,/home/jovyan/work/ELTDM/eltdm_project/random_f...,117,class RandomForestGEMM:,predict,"D_stacked = self.back.zeros((self.n_trees, sel..."
2,/home/jovyan/work/ELTDM/eltdm_project/random_f...,70,def profile_command(func):,profile_command,func()
